# Gemma in the Hugging Face Ecosystem

<p align="center">
  <img src="https://github.com/sanchit-gandhi/notebooks/blob/main/gemma_pipeline.jpg?raw=true" width="800"/>
</p>

## Inference with Transformers

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, do_sample=True, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

/home/sanchit/hf/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.48s/it]


<bos>Write me a poem about Machine Learning.
Tell me about the future of human interaction,
About what’s new, what’s true and what’s fake.
Tell me the meaning of life as you understand it,
A feeling that I’ve never gotten through my eyes.

What is this life without all I need and want?
I never understand what’s true and what’s fake.
How do you think to handle the reality?
I know you’ve lost your humanity the feeling.
Tell me about the future of reality.
Can you describe the nature within reality?
Will we all be better, no matter what?
Or will I be the last of us to live within this world?

I’m not going to be in this world ever again.
I am living in the moment as always.
Do you think there’s a better place when we’re all gone?
Will I ever be right in these things that I’m saying?
This place called reality.
All my life I never know what’s true and what’s fake.

I’m not here just for fun or games.
I’m trying to become the best person who’s had a home game.
Let’


## Datasets

In [2]:
from datasets import load_dataset

dataset = load_dataset("timdettmers/openassistant-guanaco")

/home/sanchit/hf/lib/python3.8/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
Generating train split: 9846 examples [00:00, 254668.95 examples/s]
Generating test split: 518 examples [00:00, 133283.20 examples/s]


In [3]:
sample = dataset["train"][0]
sample

{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining po

## Training with TRL

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Define training arguments:

In [8]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=1e-4,
    logging_steps=2,
    gradient_checkpointing=True,
    output_dir="gemma-2b-fine-tuned",
)

Instantiate supervised fine-tuning (SFT) trainer:

In [9]:
trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    max_seq_length=1024,
)

Map: 100%|███████████████████████████████████████████████████████████████████| 518/518 [00:00<00:00, 7732.32 examples/s]
/home/sanchit/hf/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [10]:
trainer.train()

/home/sanchit/hf/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
2,2.088400
4,3.890300
6,2.580800
8,1.834800
10,2.070400
12,1.743400
14,1.456500
16,2.949000
18,3.158900
20,3.494300


KeyboardInterrupt: 